## Run gather at k31
- Against merged dbs and normal dbs for all the ones we did before


In [ ]:
# index merged to rocksdb
sourmash index MAGs.all_k31.merged.rocksdb -F rocksdb -k 31 MAGs.all_k31.merged.zip

# make merged db of the gtdb one 
srun --account=ctbrowngrp -p med2 -J mergdb -t 2:00:00 -c 2 --mem=50gb --pty bash

mamba activate pangenomics_dev
sourmash scripts pangenome_merge \
/group/ctbrowngrp5/sourmash-db/gtdb-rs226/gtdb-rs226-reps.k31-sc10k.sig.rocksdb \
 -k 31 --scaled 10000 -o gtdb-rs226.merged_k31.rocksdb

sourmash index gtdb-rs226.merged_k31.rocksdb -F rocksdb -k 31 gtdb-rs226.merged_k31.sig




In [ ]:
# normal gather bc 2 dbs
srun --account=ctbrowngrp -p med2 -J calc-pres -t 6:00:00 -c 60 --mem=80gb --pty bash
# run the script
mamba activate branchwater-skipmer
snakemake --use-conda --resources mem_mb=80000 --rerun-triggers mtime \
-c 60 --rerun-incomplete -k -s gather_human.smk -n

In [ ]:
# do fastmultigather with these
sourmash scripts fastmultigather \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/resources/human_metag_fp.txt \
gtdb-rs226.merged_k31.10k.zip ../sketches/MAGs.all_k31.merged.rocksdb \
-k 31 -s 10000 -c 100 -o gather_k31/250811_fmg_human_gtdbmags.csv

sourmash scripts fastmultigather \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/resources/human_metag_fp.txt \
gtdb-rs226.merged_k31.rocksdb \
-k 31 -s 10000 -c 100 -o gather_k31/250811_fmg_human_gtdb.csv

In [ ]:

srun --account=ctbrowngrp -p med2 -J fmg -t 1:00:00 -c 100 --mem=100gb --pty bash

# do fmg against a concatenated database
sourmash scripts fastmultigather \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/resources/human_metag_fp.txt \
../../sketches/MAGs.all_k31.merged.zip -k 31 -s 10000 -c 100 -t 0 -o ../250808_fmg_MAGs_merged.k31.human.csv



In [ ]:
# ran the gather_pangenomedb.smk snakefile
# Now what to do with all outputs: Have confusing amount of scripts.. 
# i think concat with csvtk
csvtk concat *.csv > ../gtdbmags_pig.tax.summarized.csv

# then use python script
# use formula in here: f_weighted_at_rank * total_weighted_hashes * scaled
python 250818_avg_species.py \
../../results/sourmash_pangenome/tax/gtdbmags_pig.tax.summarized.csv \
../../results/sourmash_pangenome/tax/250830_tax_species.gtdbmag.summary.csv

python 250818_avg_species.py \
../../results/sourmash_pangenome/tax/gtdb_pig.tax.summarized.csv \
../../results/sourmash_pangenome/tax/250830_tax_species.gtdb.summary.csv

# same for genus, change rank to genus
python 250818_avg_species.py \
../../results/sourmash_pangenome/tax/gtdbmags_pig.tax.summarized.csv \
../../results/sourmash_pangenome/tax/250830_taxgenus.gtdbmag.summary.csv

python 250818_avg_species.py \
../../results/sourmash_pangenome/tax/gtdb_pig.tax.summarized.csv \
../../results/sourmash_pangenome/tax/250830_taxgenus.gtdb.summary.csv

python 250818_avg_species.py \
../../results/sourmash_pangenome/tax/gtdbmags_pig.tax.summarized.csv \
../../results/sourmash_pangenome/tax/250830_taxclass.gtdbmag.summary.csv

In [ ]:
# we need to add tax to the pig ones. 
# after gather ran tables plugin for presence absense
# for getting f_unique per species (with diff db)
python scripts/250813_specieslvl_funique.py

# need to concat all tax output
csvtk concat *.csv > ../gtdbmags_pig.tax.summarized.csv
 
# then use python script to get sums at each tax level. 


In [ ]:
mamba activate pangenomics_dev 

sourmash scripts gather_tables \
gather/*.gtdb.csv \
-c "intersect_bp" -p \
-o 250813_gatherout.gtdb.species.pig.csv 

sourmash scripts gather_tables \
gather/*.mags_and_gtdb.csv \
-c "intersect_bp" -p \
-o 250830_gatherout.gtdbmags.species.pig.csv 


In [ ]:
# set bp cutoff for table 
srun --account=ctbrowngrp -p bmm -J calc-pres -t 6:00:00 -c 2 --mem=40gb --pty bash

# 50 kb
mamba activate pangenomics_dev 

sourmash scripts gather_tables \
gather/*.gtdb.csv \
-c "intersect_bp" -p --filter 49999 \
-o 250902_gatherout.gtdb.species.50k.pig.csv 

sourmash scripts gather_tables \
gather/*.mags_and_gtdb.csv \
-c "intersect_bp" -p --filter 49999 \
-o 250830_gatherout.gtdbmags.species.50k.pig.csv 

# 100 kb
sourmash scripts gather_tables \
gather/*.gtdb.csv \
-c "intersect_bp" -p --filter 999999 \
-o 250902_gatherout.gtdb.species.100k.pig.csv 

sourmash scripts gather_tables \
gather/*.mags_and_gtdb.csv \
-c "intersect_bp" -p --filter 999999 \
-o 250830_gatherout.gtdbmags.species.100k.pig.csv 

In [ ]:
# try also adding tax
mamba activate pangenomics_dev 

sourmash scripts gather_tables \
gather/*.mags_and_gtdb.csv \
-t ../gtdb_pangenomedb/250903_gtdb_sraMAGs.lineages.csv \
--lineage "species" \
-c "intersect_bp" -p --filter 49999 \
-o 250903_gatherout.gtdbmags.species.50k.pig.csv 

sourmash scripts gather_tables \
gather/*.mags_and_gtdb.csv \
-c "intersect_bp" -p --filter 49999 \
-o 250830_gatherout.gtdbmags.species.50k.pig.csv 